### example:

https://www.toutiao.com/api/search/content/?

aid=24&offset=0&format=json&keyword=%E8%A1%97%E6%8B%8D&
autoload=true&count=20&cur_tab=1&from=search_tab&pd=synthesis


In [ ]:
import requests
from urllib.parse import urlencode

In [ ]:
def get_page(offset):
    params = {
        'aid': '24',
        'offset': offset,
        'format': 'json',
        'keyword': '街拍',
        'autoload': 'true',
        'count': '20',
        'cur_tab': '1',
        'from': 'search_tab',
        'pd': 'synthesis',
    }
    url = 'https://www.toutiao.com/api/search/content/?' + urlencode(params)
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
    except requests.ConnectionError:
        return None

In [ ]:
def get_json(json):
    if json.get('data'):
        for item in json.get('data'):
            title = item.get('title')
            images = item.get('image_list')
            for image in images:
                yield {
                    'image': image.get('url'),
                    'title': title
                }

In [ ]:
import os
from hashlib import md5

In [ ]:
def save_image(item):
    if not os.path.exists('ch6/{0}'.format(item.get('title'))):
        os.mkdir('ch6/{0}'.format(item.get('title')))
    try:
        response = requests.get(item.get('image'))
        if response.status_code == 200:
            file_path = 'ch6/{0}/{1}.{2}'.format(item.get('title'), md5(response.content).hexdigest(), 'jpg')
            if not os.path.exists(file_path):
                with open(file_path, 'wb') as f:
                    f.write(response.content)
                    print('Download! ', file_path)
            else:
                print('Already Downloaded', file_path)
    except requests.ConnectionError:
        print('Failed to Save Image')

In [ ]:
from multiprocessing.pool import Pool

In [ ]:
def main(offset):
    json = get_page(offset)
    for item in get_json(json):
        print(item)
        save_image(item)

In [ ]:
GROUP_START = 1
GROUP_END = 3
if __name__ =='__main__':
    pool = Pool()
    groups = ([x*20 for x in range(GROUP_START,GROUP_END+1)])
    pool.map(main,groups)
    pool.close()
    pool.join()